# Analysis of WeRateDogs Twitter

## Table of Contents
- [Introduction](#intro)
- [Gathering Data](#Gather)
- [Assessing Data](#Assess)
- [Cleaning Data](#Clean)



<a id='intro'></a>
## Introduction

WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc,WeRateDogs has over 4 million followers and has received international media coverage.
The main purpose in this project is to gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it , plus showcase them through analyses and visualizations 

<a id='Gather'></a>
## Gathering Data
In this project I gather data from a varity of sorces :
1. The WeRateDogs Twitter archive
2. The tweet image predictions,This file is hosted on Udacity's servers and should be downloaded programmatically 
3. Twitter API & JSON: Each tweet's retweet count and favorite ("like") count 


In [1]:
# Load all packages
import pandas as pd
import requests
import os
import tweepy
import json

In [2]:
#Read twitter archive csv file
twitter_archeive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archeive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
folder_name = "C:\\Users\\Ahmed Ezz\\Desktop\\Wrangle and analyze data project"

In [3]:
url= 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
with open(os.path.join(folder_name,url.split('/')[-1]),mode='wb') as file:
    file.write(response.content)
image_prediction = pd.read_csv('image-predictions.tsv',sep='\t')

In [4]:
consumer_key = 'Y6yswgPDydPmKIJeNAiFoiB1v'
consumer_secret = '5HK5Gv92YJ4Ql82x4gFUMNIvn3hK04iz5ad38BHNL09MjAXDTZ'
access_token = '1086508800398495745-J0zYjhHS45hUyFnOho3WfX0xWnuK2N'
access_token_secret = 'tkAcrrhyWuvWzhHrLmVnwlvpCIizx46k9ixYhCNkVz5cZ'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,
                wait_on_rate_limit=True,
                wait_on_rate_limit_notify=True)

In [5]:
tweets = {}
unfound_tweets = []
for tweet_id in twitter_archeive.tweet_id:
    try:
        
        tweet_status = api.get_status(tweet_id,tweet_mode='extended')
        tweets[str(tweet_id)]=tweet_status._json
        
    except Exception as e:
        unfound_tweets.append(tweet_id)     

Rate limit reached. Sleeping for: 84
Rate limit reached. Sleeping for: 156
Rate limit reached. Sleeping for: 169


In [6]:
print('The lenght of tweets:',len(tweets))
print('The lenght of unfound tweets:',len(unfound_tweets))

The lenght of tweets: 2340
The lenght of unfound tweets: 16


In [8]:
# Write tweets into a json file
with open(os.path.join(folder_name,'tweet_json.txt'), 'w') as file:
    json.dump(tweets, file)

In [5]:
# Open json file

with open(os.path.join(folder_name,'tweet_json.txt'), 'r') as json_file:
    data =  json.load(json_file)

In [8]:
tweet_dict = []
for tweet_id in data.keys():
    retweet_count = data[tweet_id]['retweet_count']
    favorite_count = data[tweet_id]['favorite_count']
    tweet_dict.append({'tweet_id':tweet_id,
                      "retweet_count":retweet_count,
                      "favorite_count":favorite_count})
    

[{'tweet_id': '892420643555336193',
  'retweet_count': 8312,
  'favorite_count': 38006},
 {'tweet_id': '892177421306343426',
  'retweet_count': 6139,
  'favorite_count': 32632},
 {'tweet_id': '891815181378084864',
  'retweet_count': 4065,
  'favorite_count': 24552},
 {'tweet_id': '891689557279858688',
  'retweet_count': 8456,
  'favorite_count': 41354},
 {'tweet_id': '891327558926688256',
  'retweet_count': 9154,
  'favorite_count': 39530},
 {'tweet_id': '891087950875897856',
  'retweet_count': 3046,
  'favorite_count': 19860},
 {'tweet_id': '890971913173991426',
  'retweet_count': 2018,
  'favorite_count': 11608},
 {'tweet_id': '890729181411237888',
  'retweet_count': 18436,
  'favorite_count': 64132},
 {'tweet_id': '890609185150312448',
  'retweet_count': 4182,
  'favorite_count': 27275},
 {'tweet_id': '890240255349198849',
  'retweet_count': 7214,
  'favorite_count': 31297},
 {'tweet_id': '890006608113172480',
  'retweet_count': 7170,
  'favorite_count': 30080},
 {'tweet_id': '88988

In [10]:
tweets_df = pd.DataFrame(tweet_dict)
tweets_df

,favorite_count,retweet_count,tweet_id
0,38006,8312,892420643555336193
1,32632,6139,892177421306343426
2,24552,4065,891815181378084864
3,41354,8456,891689557279858688
4,39530,9154,891327558926688256
5,19860,3046,891087950875897856
6,11608,2018,890971913173991426
7,64132,18436,890729181411237888
8,27275,4182,890609185150312448
9,31297,7214,890240255349198849
